In [5]:
import TorchESN as TN

In [6]:


import sys
sys.path.append('../')
import torch.nn
import numpy as np
from torchesn.nn import ESN
from torchesn import utils
import time
from sklearn import preprocessing
import matplotlib.pyplot as plt 
import plot_recurrence as plr
import pandas as pd
import DataLoad
from ipdb import set_trace
def torch_ESN(parameters): 
    device = torch.device('cuda')
    dtype = torch.double
    #HIData Mackey_glass
#     set_trace()
    [X_data,Y_data]=DataLoad.FilesLoad('Mackey_glass')
    torch.set_default_dtype(dtype)
    X_data = torch.from_numpy(X_data).to(device)
    Y_data = torch.from_numpy(Y_data).to(device)
    N1=5000
#     set_trace()
    trX = X_data[:N1]
    trY = Y_data[:N1]
    tsX = X_data[N1:]
    tsY = Y_data[N1:]
#     set_trace()
    washout = [500]
    input_size = trX.shape[2]
    output_size = 1
    hiddensize = int(parameters[0]*500)
    numlayers=int(parameters[1]*10)
#     w_ih_scale=parameters[2]
      
    loss_fcn = torch.nn.MSELoss()

    if __name__ == "__main__":
        start = time.time()

        # Training
#         set_trace()
        trY_flat = utils.prepare_target(trY.clone(), [trX.size(0)], washout)
    #     for numlayers in range(1,40,10):
    #         for hiddensize in range(20,500,100):
#         set_trace()
        model = ESN(input_size, hidden_size=hiddensize, output_size=output_size, num_layers=numlayers)
        model.to(device)
#         set_trace()
        model(trX, washout, None, trY_flat)
#         model.fit()
        output, hidden = model(trX, washout)
    
#         print("Training error:", loss_fcn(output, trY[washout[0]:]).item())

            # Test
#         set_trace()
        output, hidden = model(tsX, [0], hidden)
#         set_trace()
        if loss_fcn(output, tsY).item()<1e-11:
            output1=output.reshape(shape=(len(output.tolist()),)).tolist()
            output1=np.array(output1)
            set_trace()
    #                 scaler = preprocessing.StandardScaler()
    #                 output1 = scaler.fit_transform(output1)
            rec = plr.rec_plot(output1)
            plt.imshow(rec)
            #'../Results/RecurrencePlots'+
            plt.savefig(str(numlayers)+'_'+str(hiddensize)+'.png',dpi=600)
            plt.show()
     # 递归图
#         tsY1=tsY.reshape(shape=(len(tsY.tolist()),)).tolist()
#         tsY1=np.array(tsY1)
#         rectsY = plr.rec_plot(tsY1)
#         plt.imshow(rectsY)
#         plt.savefig('../Results/RecurrencePlotsend'+str(numlayers)+'_'+str(hiddensize)+'.png',dpi=600)
#         plt.show()
        

        print("Test error:", loss_fcn(output, tsY).item())
#         print("Ended in", time.time() - start, "seconds.")
#         preds=[]
#         testD=[]
#         画图
#         for i in range(500):
#             preds.append(output.tolist()[i][0])
#             testD.append(tsY[i][0])
#         plt.plot(preds, color='red')
#         plt.plot(testD,'--' )
#         plt.plot()
#         plt.show()
#         set_trace()
        return loss_fcn(output, tsY).item()



In [7]:
def euclid_dist(x,y):
    temp = 0
    for i,j in zip(x,y):
        temp += (i-j)**2
        final = np.sqrt(temp)
    return final

In [ ]:
#500 iterations for 30 particle
%pdb
import numpy as np
from numpy import random as rnd
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
import sys
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
max_iters = 10
c1 = 2
c2 = 2
num_particles = 5
g0 = 1
hidden_nodes = 15  
dim = 3*num_particles
w=2;                 
wMax=0.9            
wMin=0.5              
current_fitness = np.zeros((num_particles,1))
gbest = np.zeros((1,dim))
gbest_score = float('inf')
# input_nodes=1,
# hidden_nodes=500
# output_nodes=1
# # weights1 = np.zeros((input_nodes, hidden_nodes))
# weights2 = np.zeros((hidden_nodes, output_nodes))
# biases1 = np.zeros((1, hidden_nodes))
# biases2 = np.zeros((1, output_nodes))
convergence = np.zeros(max_iters)
alpha = 20
epsilon = 1

class Particle:
    pass

#all particle initialized
particles = []
# set_trace()
for i in range(num_particles):
    p = Particle()
    p.params = np.array([rnd.rand() for i in range(dim)])
    p.fitness = rnd.rand()
    p.velocity = 0.3*rnd.randn(dim)
    p.res_force = rnd.rand()
    p.acceleration = rnd.randn(dim)
    p.force = np.zeros(dim)
    p.id = i
    particles.append(p)
#training 
for i in range(max_iters):
    print(i)
    # gravitational constant
    g = g0*np.exp((-alpha*i)/max_iters)
    # calculate mse
    cf = 0
    parameters=np.abs(rnd.randn(1,3)).tolist()[0]
    for p in particles:
        
        fitness = 0
        y_train = 0
#         for t in trX:  #ith particle output compare with target , find fitness
#         set_trace()
        [fitness, out] = TN.torch_ESN(parameters)
#         set_trace()
#         fitness = fitness/X.shape[0]
        current_fitness[cf] = fitness
        cf += 1
        
        if gbest_score > fitness:
            gbest_score = fitness
            gbest = p.params
    
    best_fit = min(current_fitness)
    worst_fit = max(current_fitness)
    
    for p in particles:
        p.mass = (current_fitness[particles.index(p)]-0.99*worst_fit)/(best_fit-worst_fit)
    
    for p in particles:
        p.mass = p.mass*5/sum([p.mass for p in particles])
    
    
    # gravitational force
    for p in particles:
        for x in particles[particles.index(p)+1:]:
            p.force = (g*(x.mass*p.mass)*(p.params - x.params))/(euclid_dist(p.params,x.params))
    
    # resultant force
    for p in particles:
        p.res_force = p.res_force+rnd.rand()*p.force
    
    # acceleration
    for p in particles:
        p.acc = p.res_force/p.mass
    
    w = wMin-(i*(wMax-wMin)/max_iters)
    
    # velocity
    for p in particles:
        p.velocity = w*p.velocity+rnd.rand()*p.acceleration+rnd.rand()*(gbest - p.params)
    
    # position
    for p in particles:
        p.params = p.params + p.velocity
    
    convergence[i] = gbest_score
    sys.stdout.write('\rPSOGSA is training ESN (Iteration = ' + str(i+1) + ', MSE = ' + str(gbest_score) + ')')
    sys.stdout.flush()
return convergence, p.params 

Automatic pdb calling has been turned ON
0


AttributeError: 'float' object has no attribute 'size'

> /git/hilbertNN/Mypytorch-esn/torchesn/nn/echo_state_network.py(192)forward()
    190                 batch_size = output.size(1)
    191 #                 set_trace()
--> 192                 X = torch.ones(target.size(0), 1 + output.size(2), device=target.device)
    193                 row = 0
    194                 for s in range(batch_size):

